In [6]:
import psi4
psi4.set_output_file('output.dat', False)

PosixPath('output.dat')

In [7]:
#! Sample HF/cc-pVDZ H2O Computation
psi4.set_memory('500 MB')

500000000

In [8]:
%%time
h2o = psi4.geometry("""
O
H 1 0.96
H 1 0.96 2 104.5
""")

CPU times: user 835 µs, sys: 0 ns, total: 835 µs
Wall time: 844 µs


In [9]:
%%time
psi4.energy('scf/cc-pvdz')

CPU times: user 301 ms, sys: 10.2 ms, total: 311 ms
Wall time: 331 ms


-76.0266327350902

In [17]:
#! Sample UHF/6-31G** CH2 Computation
R = 1.075
A = 133.93

In [18]:
ch2 = psi4.geometry("""
0 3
C
H 1 {0}
H 1 {0} 2 {1}
""".format(R, A)
)

In [19]:
%%time
psi4.set_options({'reference': 'uhf'})
psi4.energy('scf/6-31g**')

CPU times: user 291 ms, sys: 6.06 ms, total: 297 ms
Wall time: 298 ms


-38.92533462893707

In [21]:
%%time
scf_e, scf_wfn = psi4.frequency('scf/cc-pvdz', molecule=h2o, return_wfn=True)

CPU times: user 1.27 s, sys: 24 ms, total: 1.29 s
Wall time: 1.3 s


In [22]:
# Example SAPT computation for ethene*ethyne (*i.e.*, ethylene*acetylene).
# Test case 16 from S22 Database

dimer = psi4.geometry("""
0 1
C   0.000000  -0.667578  -2.124659
C   0.000000   0.667578  -2.124659
H   0.923621  -1.232253  -2.126185
H  -0.923621  -1.232253  -2.126185
H  -0.923621   1.232253  -2.126185
H   0.923621   1.232253  -2.126185
--
0 1
C   0.000000   0.000000   2.900503
C   0.000000   0.000000   1.693240
H   0.000000   0.000000   0.627352
H   0.000000   0.000000   3.963929
units angstrom
""")

In [23]:
%%time
psi4.set_options({'scf_type': 'df',
                  'freeze_core': True})

CPU times: user 53 µs, sys: 3 µs, total: 56 µs
Wall time: 58.7 µs


In [24]:
%%time
psi4.energy('sapt0/jun-cc-pvdz', molecule=dimer)

CPU times: user 8.64 s, sys: 190 ms, total: 8.83 s
Wall time: 8.86 s


-0.0022355918478786825

In [25]:
#! Example potential energy surface scan and CP-correction for Ne2

ne2_geometry = """
Ne
--
Ne 1 {0}
"""

Rvals = [2.5, 3.0, 4.0]

psi4.set_options({'freeze_core': True})

In [27]:
%%time
# Initialize a blank dictionary of counterpoise corrected energies
# (Need this for the syntax below to work)

ecp = {}

for R in Rvals:
    ne2 = psi4.geometry(ne2_geometry.format(R))
    ecp[R] = psi4.energy('ccsd(t)/aug-cc-pvdz', bsse_type='cp', molecule=ne2)


CPU times: user 8.31 s, sys: 2.69 s, total: 11 s
Wall time: 11.4 s


In [28]:
%%time
# Prints to screen
print("CP-corrected CCSD(T)/aug-cc-pVDZ Interaction Energies\n\n")
print("          R [Ang]                 E_int [kcal/mol]       ")
print("---------------------------------------------------------")
for R in Rvals:
    e = ecp[R] * psi4.constants.hartree2kcalmol
    print("            {:3.1f}                        {:1.6f}".format(R, e))

CP-corrected CCSD(T)/aug-cc-pVDZ Interaction Energies


          R [Ang]                 E_int [kcal/mol]       
---------------------------------------------------------
            2.5                        0.758605
            3.0                        0.015968
            4.0                        -0.016215
CPU times: user 294 µs, sys: 0 ns, total: 294 µs
Wall time: 281 µs


In [29]:
%%time
# Prints to output.dat
psi4.core.print_out("CP-corrected CCSD(T)/aug-cc-pVDZ Interaction Energies\n\n")
psi4.core.print_out("          R [Ang]                 E_int [kcal/mol]       \n")
psi4.core.print_out("---------------------------------------------------------\n")
for R in Rvals:
    e = ecp[R] * psi4.constants.hartree2kcalmol
    psi4.core.print_out("            {:3.1f}                        {:1.6f}\n".format(R, e))

CPU times: user 98 µs, sys: 0 ns, total: 98 µs
Wall time: 101 µs
